In [44]:
from sklearn.datasets import load_wine
from datetime import timedelta
import pandas as pd
import numpy as np

In [45]:
read_data_file = pd.read_csv('..\dataFrame_flink.csv')
df = pd.DataFrame(read_data_file)

In [46]:
df.columns

Index(['commit_befor ', 'commit_after', 'cal_smell', 'dispen_bf', 'object_bf',
       'bloater_bf', 'dispen_at', 'object_at', 'bloater_af', 'open_time',
       'closed_time', 'cal_time', 'changed_file', 'additions', 'deletions',
       'dev', 'cal_time_binaly'],
      dtype='object')

In [47]:
X = df[['commit_befor ', 'dispen_bf', 'object_bf', 'bloater_bf', 'open_time',]]
y = df['cal_time_binaly']

In [48]:
X = X.dropna()
y = y.dropna()

In [49]:
X.shape

(30, 5)

In [50]:
y.shape

(30,)

In [51]:
X_train ,X_test ,y_train , y_test = X[:20], X[20:],y[:20],y[20:]

In [52]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [53]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf , X , y , cv=5 , scoring= "accuracy")

array([0.5       , 0.33333333, 0.33333333, 0.16666667, 0.66666667])

In [54]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf,X,y,cv=5)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1], dtype=int64)

In [55]:
y_pred_test =cross_val_predict(clf,X_test,y_test,cv=5)

C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [56]:
confusion_matrix(y,y_pred)

array([[ 8,  7],
       [11,  4]], dtype=int64)

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred))
print('recall_score = ', recall_score(y,y_pred))
print('f1_score',f1_score(y,y_pred))

precision =  0.36363636363636365
recall_score =  0.26666666666666666
f1_score 0.30769230769230765


In [58]:
frame={"y":y,"y_pred": y_pred}
pd.DataFrame(frame).sum()

y         15
y_pred    11
dtype: int64

In [59]:
from sklearn.model_selection import GridSearchCV

parameters = {
  
    'C': [0.01 , 0.1 ,1],
    'gamma' : ['scale' , 'auto'],
    'random_state':[2]
  
}
svc = SVC()
clf = GridSearchCV(svc , parameters , cv=3 , scoring='f1_macro')
clf.fit(X_test,y_test)

GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': [0.01, 0.1, 1], 'gamma': ['scale', 'auto'],
                         'random_state': [2]},
             scoring='f1_macro')

In [60]:
y_pred_clf = clf.best_estimator_.predict(X)

In [61]:
clf.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [62]:
from sklearn.metrics import precision_score, recall_score, f1_score
print('precision = ', precision_score(y,y_pred_clf ,average='macro'))
print('recall_score = ', recall_score(y,y_pred_clf ,average='macro'))
print('f1_score',f1_score(y,y_pred_clf ,average='macro'))

precision =  0.25
recall_score =  0.5
f1_score 0.3333333333333333


C:\Users\Beauz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np

def f_importances(coef, names):
    
    names = np.arange(1)
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = X.columns
svm1 = svm.SVC(kernel='linear',C=1)
svm1.fit(X, y)

SVC(C=1, kernel='linear')

In [64]:
svm1.coef_

array([[-0.50518342,  0.48324321,  0.36899649,  1.17543127, -0.28573192]])

In [65]:
import itertools

In [66]:
itertools.combinations(iterable = X , r = 2)

In [67]:
def combinations(iterable = X , r=2):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [68]:
print([itertools.combinations(iterable = X , r=2)])

In [69]:
print([''.join(i) for i in itertools.combinations(X, 2)])

['commit_befor dispen_bf', 'commit_befor object_bf', 'commit_befor bloater_bf', 'commit_befor open_time', 'dispen_bfobject_bf', 'dispen_bfbloater_bf', 'dispen_bfopen_time', 'object_bfbloater_bf', 'object_bfopen_time', 'bloater_bfopen_time']
